# NIPS 2017 Figure 4

In [ ]:
%run -i NIPS2017.ipynb

# FIGURE and GRID
FIG_HEIGHT_MM = 35
FIG_WIDTH_MM = FIG_WIDTH_MM  # set in NIPS2017 notebook to a default value for all figures
FIG_N_ROWS = 1
ROW_1_NCOLS = 4
ROW_1_HEIGHT_MM = FIG_HEIGHT_MM / FIG_N_ROWS
ROW_1_WIDTH_COL_1_MM = FIG_WIDTH_MM / ROW_1_NCOLS
ROW_1_WIDTH_COL_2_MM = FIG_WIDTH_MM / ROW_1_NCOLS
ROW_1_WIDTH_COL_3_MM = FIG_WIDTH_MM / ROW_1_NCOLS
ROW_1_WIDTH_COL_4_MM = FIG_WIDTH_MM / ROW_1_NCOLS

# PATHS
PANEL_A = PATH_DROPBOX_FIGS + 'fig4_a.svg'
PANEL_B = PATH_DROPBOX_FIGS + 'fig4_b.svg'
PANEL_D = PATH_DROPBOX_FIGS + 'fig4_c.svg'
PANEL_D = PATH_DROPBOX_FIGS + 'fig4_d.svg'

## Row 1

Generate bad simulation data

In [ ]:
from lfmods.autapse import AutapseSimulator

sim = AutapseSimulator(seed=1, pilot_samples=0.,cached_sims=False, cached_pilot=False)
true_params = sim.true_params
n_params = sim.n_params

init = sim.init
t = sim.t
dt = sim.dt
duration = np.max(t)
I = sim.I_obs

params = np.array([0.98]).reshape(-1,1)
hh = bm = sim.bm.Autapse(init, params.reshape(1,-1), seed=1)
states_good_autapse = hh.sim_time(dt, t, I)[:,0]

params = np.array([1.1]).reshape(-1,1)
hh = bm = sim.bm.Autapse(init, params.reshape(1,-1), seed=1)
states_bad_autapse = hh.sim_time(dt, t, I)[:,0]

Autapse simulations

In [ ]:
prefixes = ['lr_s1', 'no_lr_s1']
verbose = False

bad_simulations = {}

for prefix in prefixes:

    dists, infos, losses, nets, posteriors, sims = io.load_prefix(dirs['dir_nets_autapse'], prefix)
    sim = io.last(sims)
    obs_stats = sim.obs
    gt = sim.true_params
    num_param_inf = len(gt)
    y_obs = sim.obs_trace.reshape(-1,1)
    labels_params = sim.labels_params
    t = sim.t
    I = sim.I_obs.reshape(1,-1)
    posterior = io.last(posteriors)

    bs = []
    for key, info in infos.items():
        total = info['n_samples']+info['bad_data_round']
        bad = info['bad_data_round']
        bs.append(bad/total * 100)
        
        if info['bad_data_predictor'] is not None and verbose:
            print('{} : decision threshold : {:.3f}'.format(key, ((0.5-info['bad_data_predictor'].intercept_)/info['bad_data_predictor'].coef_)[0,0]))
            
    bad_simulations[prefix] = np.array(bs)

if verbose:
    plt.plot(bad_simulations[prefixes[0]], color='b')
    plt.hold(True)
    plt.plot(bad_simulations[prefixes[1]], color='r')
    plt.show()

Missing data Hodgkin-Huxley

In [ ]:
from lfmods.hh import HHSimulator

sim = HHSimulator(seed=100,pilot_samples=0.,cached_sims=False, cached_pilot=False)

# ground-truth parameters
true_params = sim.true_params
n_params = sim.n_params

init = sim.init
t = sim.t
dt = sim.dt
duration = np.max(t)
I = sim.I_obs

# prior over parameters
sim_prior = sim.sim_prior
sim_prior()
prior_min = sim.prior_min
prior_max = sim.prior_max

# if parameters in transformed space, inverse transform function
param_invtransform = sim.param_invtransform

bm = sim.bm

# simulate missing feature trace
fact = .9
new_prior_min = fact*prior_min
new_prior_max = fact*prior_max
np.random.seed(123)
z = np.random.rand(n_params)
params = param_invtransform( np.multiply(new_prior_max - new_prior_min,z) + new_prior_min )
hh = bm.HH(init, params.reshape(1,-1),seed=230)
states_mf = hh.sim_time(dt, t, I)[:,0]

# simulate trace with spikes
fact = 1.
new_prior_min = fact*prior_min
new_prior_max = fact*prior_max
np.random.seed(123)
z = np.random.rand(n_params)
params = param_invtransform( np.multiply(new_prior_max - new_prior_min,z) + new_prior_min )
hh = bm.HH(init, params.reshape(1,-1),seed=230)
states_spikes = hh.sim_time(dt, t, I)[:,0]

### Panel A

In [ ]:
fig_inches = (SCALE_IN*mm2inches(0.9*ROW_1_WIDTH_COL_1_MM), SCALE_IN*mm2inches(0.95*ROW_1_HEIGHT_MM))

with mpl.rc_context(fname=MPL_RC):
    fig = plt.figure(figsize=fig_inches)

    #gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[4, 1])
    #ax = plt.subplot(gs[0])
    plt.plot(t, states_good_autapse, color=COL['GT'], lw=3)
    plt.plot(t, states_bad_autapse, color='orange', lw=3)
    plt.legend(['observed data', 'bad simulation'])
    plt.xlabel('time (ms)')
    plt.ylabel('rate (Hz)')
    ax = plt.gca()
    ax.set_xlim([0, duration])
    #ax.set_xticks([])
    plt.yscale('log', nonposy='clip')
    plt.ylim([0, 1e4])

    """ hiding the input
    ax = plt.subplot(gs[1])
    plt.plot(t,I,'r', lw=3)
    plt.xlabel('')
    plt.ylabel('input (mA)')
    ax.set_xlim([0, duration])
    ax.set_xticks([0, duration/2, duration])
    ax.set_yticks([0, 1.1*np.max(I)])
    ax.yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.1f'))
    ax.xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%d'))

    """
    
    plt.savefig(PANEL_A, facecolor=plt.gcf().get_facecolor(), transparent=True)  # the figure is saved as svg
    plt.close()

svg(PANEL_A)

panel = st.fromfile(PANEL_A)
width, height = panel.get_size()
print('size of svg : {}'.format((width, height)))
print('aspect ratio : {}'.format(get_num(width) / get_num(height)))
print('desired ratio : {}'.format(ROW_1_WIDTH_COL_1_MM / ROW_1_HEIGHT_MM))

### Panel B

In [ ]:
fig_inches = (SCALE_IN*mm2inches(ROW_1_WIDTH_COL_2_MM), SCALE_IN*mm2inches(0.95*ROW_1_HEIGHT_MM))

with mpl.rc_context(fname=MPL_RC, rc={'text.usetex': False, 'legend.handlelength': 0}):
    fig = plt.figure(figsize=fig_inches)

    plt.plot(bad_simulations[prefixes[0]], color=COL['SNPE'])
    plt.hold(True)
    plt.plot(bad_simulations[prefixes[1]], color='k')

    plt.ylabel('bad data simulations (%)')
    plt.xlabel('round')
  
    plt.gca().text(6, 45, 'baseline')

    legend_fontdict = {'color': COL['SNPE']}
    plt.gca().text(6, 37, 'predictive\nmodel', legend_fontdict)
    
    plt.ylim([0, 50])
    #ax.set_xlim([0, duration])
    #ax.set_xticks([])

    plt.savefig(PANEL_B, facecolor='None', transparent=True)
    plt.close()

svg(PANEL_B)

panel = st.fromfile(PANEL_B)
width, height = panel.get_size()
print('size of svg : {}'.format((width, height)))
print('aspect ratio : {}'.format(get_num(width) / get_num(height)))
print('desired ratio : {}'.format(ROW_1_WIDTH_COL_1_MM / ROW_1_HEIGHT_MM))

### Panel C

In [ ]:
fig_inches = (SCALE_IN*mm2inches(ROW_1_WIDTH_COL_1_MM), SCALE_IN*mm2inches(0.95*ROW_1_HEIGHT_MM))

with mpl.rc_context(fname=MPL_RC):
    fig = plt.figure(figsize=fig_inches)

    gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[4, 1])
    ax = plt.subplot(gs[0])
    plt.plot(t, states_spikes, color='k', alpha=0.5, lw=2)
    plt.plot(t, states_mf, color=COL['GT'], lw=2)
    

    plt.ylabel('voltage (mV)')

    ax.set_xlim([0, duration])
    ax.set_xticks([])
    ax.set_yticks([-80, -20, 40])

    ax = plt.subplot(gs[1])
    plt.plot(t,I,'r', lw=2)
    plt.xlabel('')
    plt.ylabel('input (mA)')

    ax.set_xlim([0, duration])
    ax.set_xticks([0, duration/2, duration])
    ax.set_yticks([0, 1.1*np.max(I)])
    ax.yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.1f'))
    ax.xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%d'))
    plt.xlabel('time (ms)')

    plt.savefig(PANEL_C, facecolor=plt.gcf().get_facecolor(), transparent=True)
    plt.close()
    
svg(PANEL_C)

panel = st.fromfile(PANEL_C)
width, height = panel.get_size()
print('size of svg : {}'.format((width, height)))
print('aspect ratio : {}'.format(get_num(width) / get_num(height)))
print('desired ratio : {}'.format(ROW_1_WIDTH_COL_1_MM / ROW_1_HEIGHT_MM))

### Panel D

In [ ]:
try:
    svg(PANEL_D)
except:
    pass

## Compose figure

In [ ]:
fig = create_fig(FIG_WIDTH_MM, FIG_HEIGHT_MM)

yoffset = -1.2
xoffset = -1.
fig = add_svg(fig, PANEL_A, 0 + xoffset, 0 + yoffset + 1)
fig = add_svg(fig, PANEL_B, ROW_1_WIDTH_COL_1_MM + xoffset, 0 + 1.3 + yoffset)
fig = add_svg(fig, PANEL_C, ROW_1_WIDTH_COL_1_MM + ROW_1_WIDTH_COL_2_MM + xoffset, 0 + yoffset + 1)
#fig = add_svg(fig, PANEL_D, ROW_1_WIDTH_COL_1_MM + ROW_1_WIDTH_COL_2_MM + ROW_1_WIDTH_COL_3_MM + xoffset, 0 + yoffset)

yoffset   = 2.3
fig = add_label(fig, 'A', 0, yoffset)
fig = add_label(fig, 'B', ROW_1_WIDTH_COL_1_MM, yoffset)
fig = add_label(fig, 'C', ROW_1_WIDTH_COL_1_MM + ROW_1_WIDTH_COL_2_MM, yoffset)
fig = add_label(fig, 'D', ROW_1_WIDTH_COL_1_MM + ROW_1_WIDTH_COL_2_MM + ROW_1_WIDTH_COL_3_MM, yoffset)

if False:
    fig = add_grid(fig, 2, 2)
    fig = add_grid(fig, 160/3, 10, font_size_px=0.0001)

PATH_SVG = PATH_DROPBOX_FIGS + 'fig4.svg'
fig.save(PATH_SVG)

svg(PATH_SVG)

!$INKSCAPE --export-pdf $PATH_DROPBOX_FIGS/fig4.pdf $PATH_SVG